In [1]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [2]:
#visualise maze:
maze1 = ["SFF", "FHF", "FGF"]

desc = maze1
mazeSize = [len(desc),len(desc[0])]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        

        
giftState = -1
gift_found = False
for i in range(len(desc)):
    if gift_found:
        break
    for j in range(len(desc[i])):
        giftState += 1
        if desc[i][j] == 'G':
            gift_found = True
            break
            
print(giftState)
print(statePositions)

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

7
[[0, 1, 2], [3, 4, 5], [6, 7, 8]]


In [3]:
def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 0
    
def getPosition(state):
    for i in range(len(statePositions)):
        for j in range(len(statePositions[i])):
            if statePositions[i][j] == state:
                return i, j
            
def calcReward(state, nextState):
    y1, x1 = getPosition(state)
    y2, x2 = getPosition(nextState)
    y3, x3 = getPosition(giftState)
    
    currentDist = (((y3 - y1)**2)+((x3 - x1)**2))**0.5
    nextDist = (((y3 - y2)**2)+((x3 - x2)**2))**0.5
    
    changeInDist = currentDist-nextDist
    if changeInDist > 0:
        return changeInDist/2
    else:
        return changeInDist

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 0:
        return [1,2]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    return possibleMoves

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]

        maxValue = max(qValues.values())
        minValue = min(qValues.values())
        count_max = sum(1 for value in qValues.values() if value == maxValue)
        count_min = sum(1 for value in qValues.values() if value == minValue)

        if count_max > 1 and count_max < len(possMoves):
            nextMove = random.choice([move for move in possMoves if qValues[move] != minValue])
        elif count_max == len(possMoves):
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def pathFound():
    currentState = 0
    for i in range(mazeSize[0]*mazeSize[1]-1):
        bestDirection = None
        if max(qTable_1[currentState]) > 0:
            bestDirection = qTable_1[currentState].index(max(qTable_1[currentState]))
        newState = 0
        if bestDirection == 0 and 0 in calcPossibleMoves(currentState):
            newState = currentState - 1
        elif bestDirection == 1 and 1 in calcPossibleMoves(currentState):
            newState = currentState + mazeSize[1]
        elif bestDirection == 2 and 2 in calcPossibleMoves(currentState):
            newState = currentState + 1
        elif bestDirection == 3 and 3 in calcPossibleMoves(currentState):
            newState = currentState - mazeSize[1]

        if newState == giftState:
            return True
        currentState = newState
    return False

def updateTable_sarsa(direction, nextState, reward):
    global qTable_1
    global currentState
    didConverge = False
    nextDirection = nextStep(nextState)
    updated = qTable_1[currentState][direction] + alpha*(reward + (gamma*qTable_1[nextState][nextDirection]) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ, nextDirection

In [10]:
#SARSA-----------------------------------------------------------------------

epsilonValue = 0.35
alpha = 0.6
gamma = 0.7
qTable_1 = {}
currentState = 0

convergenceThresh = 0.01

maxEpisodes = 1000
currentEpisode = 1
converged = False
revistPenalty = -0.25
startOfEpisode = True
direction = None

firstPath = []
firstPathEpisode = []
foundFirstPath = False

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0
resetVisited()

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    global startOfEpisode
    global currentSteps
    global Direction
    global firstPath
    global foundFirstPath
    if converged:
        break
        
    if startOfEpisode:
        direction = nextStep(currentState)
        startOfEpisode = False
    nextState, reward, terminated, truncated, info = env.step(direction)
    if not foundFirstPath:
        firstPath.append(nextState)
    
    if terminated:
        if reward < 1:
            reward = -10
        else:
            if not foundFirstPath:
                foundFirstPath = True
                firstPathEpisode = [currentEpisode, firstPath]
            reward = 10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
        if timesVisited[nextState] > 0:
            reward += revistPenalty*timesVisited[nextState]
    
    converged, changeInQ, nextDirection = updateTable_sarsa(direction, nextState, reward)
    direction = nextDirection

    if terminated or truncated or converged:
        observation, info = env.reset()
        if not converged:
            currentEpisode += 1
            startOfEpisode = False
            if not foundFirstPath:
                firstPath = []
            resetVisited()

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 20/1000
Time: 31.825 sec
Q-Table:
0: [0, 4.050427572459555, 1.2694725329421868, 0]
1: [1.02987134426452, -9.36, -0.14164078649987388, 0]
2: [0, 0.24655632453800838, 0, 0]
3: [0, 6.5720162101736115, 0, -1.4663136990547407]
4: [0, -0.23846086502124308, -0.6, 0.4286485489246589]
5: [0, 0.023969696196699974, 0, 0]
6: [0, 0, 9.966166046162456, 1.066009696524796]
7: [-1.5325443958126777, 0, -1.5269694392336879, -0.04811917723538106]
8: [5.4365108696700535, 0, 0, -0.3463372285648471]
change in Q: 0.00023
31.821 seconds to converge


In [11]:
print(firstPathEpisode)

[1, [3, 6, 3, 0, 3, 6, 7]]


In [12]:
env.close()